Self check to write a small delta table and then read it from a new spark session.

In [ ]:
result = {
    "area": "spark",
    "description": "Check that spark can use iceberg tables.",
    "passed": False,
    "message": "",
    "plugin": "spark",
}

In [ ]:
from pyspark.sql import SparkSession
from datetime import datetime

# Native SparkSession with Polaris + MinIO
spark = (
    SparkSession.builder
    .appName("IcebergPolarisTest")
    .master("spark://spark-master:7077")
    .config("spark.driver.extraClassPath", "/opt/freeds/spark/jars/*")
    .config("spark.executor.extraClassPath", "/opt/freeds/spark/jars/*")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.freeds_catalog", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.freeds_catalog.type", "rest")
    .config("spark.sql.catalog.freeds_catalog.uri", "http://polaris:8181")
    .config("spark.sql.catalog.freeds_catalog.warehouse", "s3a://dwh/warehouse/")
    .config("spark.hadoop.fs.s3a.access.key", "kbmB5u0xCQs97d0vJxjx")
    .config("spark.hadoop.fs.s3a.secret.key", "Tjq8PW7h0GU7rhiMrNMRrvCfpOhfIgvV8dwEhyIJ")
    .config("spark.hadoop.fs.s3a.endpoint", "http://s3-minio:9900")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .getOrCreate()
)
print(spark.catalog.listCatalogs())
# Use the Polaris catalog first
spark.sql("USE CATALOG freeds_catalog")

# Create a namespace/schema inside the catalog
spark.sql("CREATE NAMESPACE IF NOT EXISTS test_db")

# Drop table if exists
spark.sql(f"DROP TABLE IF EXISTS {table_name}")

# Create Iceberg table in Polaris catalog
spark.sql(
    f"""
    CREATE TABLE {table_name} (
        id INT,
        name STRING,
        created_at TIMESTAMP
    )
    USING iceberg
    """
)

# Sample data
data = [(1, "Alice", datetime.now()), (2, "Bob", datetime.now())]
df = spark.createDataFrame(data, ["id", "name", "created_at"])

# Write data using DataFrameWriterV2
df.writeTo(table_name).append()

# Read back
df_read = spark.table(table_name)
df_read.show()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/12 19:03:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/09/12 19:03:50 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
log4j:WARN No appenders could be found for logger (com.amazonaws.AmazonWebServiceClient).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


[CatalogMetadata(name='spark_catalog', description=None)]


Py4JJavaError: An error occurred while calling o51.sql.
: org.apache.iceberg.exceptions.RESTException: Unable to process: HTTP 404 Not Found
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:248)
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:212)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:215)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:299)
	at org.apache.iceberg.rest.BaseHTTPClient.get(BaseHTTPClient.java:77)
	at org.apache.iceberg.rest.RESTSessionCatalog.fetchConfig(RESTSessionCatalog.java:1021)
	at org.apache.iceberg.rest.RESTSessionCatalog.initialize(RESTSessionCatalog.java:202)
	at org.apache.iceberg.rest.RESTCatalog.initialize(RESTCatalog.java:82)
	at org.apache.iceberg.CatalogUtil.loadCatalog(CatalogUtil.java:277)
	at org.apache.iceberg.CatalogUtil.buildIcebergCatalog(CatalogUtil.java:331)
	at org.apache.iceberg.spark.SparkCatalog.buildIcebergCatalog(Spark

In [ ]:
import json

from freeds.spark import get_spark_session, show_spark_info

try:
    spark = get_spark_session("self-check1")
    db_name = "selfcheck_db"
    table_name = f"{db_name}.selfcheck_tbl"

    # create some data in delta
    print(f"Dropping and recreating database {db_name}")
    spark.sql(f"DROP DATABASE IF EXISTS {db_name} CASCADE")
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {db_name}")
    print(f"Writing table {table_name}")
    data = spark.range(100)
    (
        data.write.option(  # .mode("overwrite")  # Options: 'overwrite', 'append', 'ignore', 'error' (default)
            "mergeSchema", "true"
        )
        .format("iceberg")  # Options: 'parquet', 'csv', 'json', 'orc', 'iceberg', 'delta' etc.
        .saveAsTable(table_name)
    )
    spark.stop()

    # read some data in delta
    spark = get_spark_session("self-check2")
    data = spark.table(table_name)
    show_spark_info(spark)
    data.show(5)

    # clean up
    spark.sql(f"DROP DATABASE IF EXISTS {db_name} CASCADE")
    spark.stop()

    result["message"] = "Executed spark cell ok."
    result["passed"] = True

except Exception as ex:
    result["message"] = str(ex)
    print(str(ex))
    result["passed"] = False

In [ ]:
print(json.dumps(result, indent=4))